In [1]:
from FlagEmbedding import FlagReranker
import os
import time
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, format_document
from langchain.prompts.prompt import PromptTemplate
import requests
import sys
from langchain_core.prompts import ChatPromptTemplate, format_document
from langchain.prompts.prompt import PromptTemplate

c:\Users\LOQ\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["GROQ_API_KEY"] = "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"  

model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} 

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_norm = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': DEVICE},
    encode_kwargs=encode_kwargs)

db3 = Chroma(persist_directory="db", embedding_function=model_norm)

In [3]:
import os
import time
import requests
from typing import List, Optional
retriever = db3.as_retriever(search_kwargs={"k": 100})
def find_first_string(obj) -> Optional[str]:
    if isinstance(obj, str):
        return obj
    elif isinstance(obj, dict):
        for v in obj.values():
            found = find_first_string(v)
            if found:
                return found
    elif isinstance(obj, list):
        for item in obj:
            found = find_first_string(item)
            if found:
                return found
    return None

def get_answer(input_text: str, max_retries: int = 10, backoff_sec: float = 2.0) -> str:
    url = 'https://api.groq.com/openai/v1/chat/completions'
    headers = {
        'Authorization': 'Bearer ' + os.environ["GROQ_API_KEY"],
        'Content-Type': 'application/json'
    }

    data = {
        "model": "llama3-70b-8192",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": input_text}
        ],
        "temperature": 0.7,
        "max_tokens": 100,
        "top_p": 0.95
    }

    for attempt in range(1, max_retries + 1):
        response = requests.post(url, headers=headers, json=data)
        print(f"Attempt {attempt} - Status: {response.status_code}")

        if response.status_code == 429:
            # Rate limit exceeded, chờ rồi thử lại
            retry_after = backoff_sec * attempt
            print(f"Rate limit reached. Retrying after {retry_after:.1f} seconds...")
            time.sleep(retry_after)
            continue

        try:
            result = response.json()
            print("Full JSON response:", result)

            if 'choices' in result and len(result['choices']) > 0:
                message = result['choices'][0].get('message')
                if message and 'content' in message:
                    return message['content']

            if 'text' in result:
                return result['text']

            if 'output' in result:
                output = result['output']
                return output if isinstance(output, str) else str(output)

            any_text = find_first_string(result)
            if any_text:
                return any_text

            return "No valid response content found."

        except Exception as e:
            return f"Error parsing response: {e}"

    return "Failed to get valid response after retries."

from langchain import PromptTemplate
from typing import Any

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def combine_documents(docs: List[Any], document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator: str = "\n\n") -> str:
    # docs có thể là List[Document] hoặc List[Tuple]
    doc_strings = []
    for doc in docs:
        # Nếu doc có attribute page_content (Document của langchain)
        if hasattr(doc, 'page_content'):
            doc_str = document_prompt.format(page_content=doc.page_content)
        elif isinstance(doc, (list, tuple)) and len(doc) > 1:
            doc_str = str(doc[1])
        else:
            doc_str = str(doc)
        doc_strings.append(doc_str)
    return document_separator.join(doc_strings)

def split_questions(text: str, delimiter: str = '?') -> List[str]:
    if delimiter not in text:
        return [text.strip()]  # Trả về nguyên văn nếu không có dấu câu hỏi
    questions = text.split(delimiter)
    cleaned_questions = []
    for q in questions:
        q = q.strip()
        if not q:
            continue
        # Lấy phần sau dấu chấm đầu tiên nếu có, rồi thêm lại dấu hỏi
        if ". " in q:
            q = q.split(". ", 1)[-1]
        cleaned_questions.append(q + delimiter)
    return cleaned_questions
def combine_documents_2(docs, document_separator="\n\n"):
    combined_docs = []
    for doc in docs:
        combined_docs.append(doc[1])
    return document_separator.join(combined_docs)


In [ ]:
# Sử dụng Retriever (thường)
def _unique_documents(documents):
    return [doc for i, doc in enumerate(documents) if doc not in documents[:i]]
with open('Data_test/test_questions.csv', 'r') as file:
    questions = file.readlines()
for question in questions:
    docs = retriever.invoke(question)

    prompt_start = """You are an assistant for question-answering tasks and the questions are related to the University of Pittsburgh and Carnegie Mellon University (CMU). Do not exceed one sentence for the answer. Do not be verbose when generating the answer. Give out the answer directly even if it does not form a coherent sentence."""
    context = combine_documents(docs[:10])
    input_text = prompt_start + "Question: " + question + "Context: " + context + "Answer: "
    time.sleep(2)
    print(question)

    answer = get_answer(input_text)
    with open('system_output_1.txt', 'a', encoding='utf-8') as output_file:
        output_file.write(f'{answer}\n')

c:\Users\LOQ\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Question

Attempt 1 - Status: 200
Full JSON response: {'id': 'chatcmpl-abf8c51e-81e8-43a0-bf97-2b0a32a752c7', 'object': 'chat.completion', 'created': 1747643100, 'model': 'llama3-70b-8192', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'This appears to be unrelated to the University of Pittsburgh or Carnegie Mellon University.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'queue_time': 0.220612136, 'prompt_tokens': 166, 'prompt_time': 0.005281152, 'completion_tokens': 16, 'completion_time': 0.053082765, 'total_tokens': 182, 'total_time': 0.058363917}, 'usage_breakdown': {'models': None}, 'system_fingerprint': 'fp_dd4ae1c591', 'x_groq': {'id': 'req_01jvktavk0fvpvs5ktphnjz9v3'}}
"What bank, which is the 5th largest in the US, is based in Pittsburgh?"

Attempt 1 - Status: 429
Rate limit reached. Retrying after 2.0 seconds...
Attempt 2 - Status: 429
Rate limit reached. Retrying after 4.0 seconds...
Attempt 3 - Status: 429
Rate limit reached. Retrying afte